# Outfit Completion via Cosine Similarity + Regression Model + LLaVA Scoring
- Selects most similar items from each cluster using cosine similarity
- Uses regression model to pick the best outfit
- Uses LLaVA to rescore the final recommendation
- Outputs average LLaVA score for all test anchors

In [1]:
# ✅ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip uninstall -y numpy transformers
!pip install numpy==1.24.4 --no-cache-dir --force-reinstall
!pip install git+https://github.com/huggingface/transformers.git
!pip install accelerate bitsandbytes xformers pandas openpyxl --upgrade

# 🔁 Restart runtime to fully apply changes
import os
os.kill(os.getpid(), 9)

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: transformers 4.50.3
Uninstalling transformers-4.50.3:
  Successfully uninstalled transformers-4.50.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 318.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, which is not installed.
peft 0.14.0 requires transformers, which is not installed.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
blosc2 3.2.1 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
thin

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-kzgopqhg
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-kzgopqhg
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
^C


In [1]:
# Install HuggingFace Transformers from the latest GitHub repo
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-3qsmhyfo
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-3qsmhyfo
  Resolved https://github.com/huggingface/transformers.git to commit 953196a43dae6a3c474165fba7d215fcbc7b7730
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.52.0.dev0-py3-none-any.whl size=11132583 sha256=41785f63abd5f3837d17af0433155e13b9a03254e3539872640237b9ef9e31da
  Stored in directory: /tmp/pip-ephem-wheel-cache-fauxqse8/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers


In [2]:
!pip install jax --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 92.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.5.1
    Uninstalling jaxlib-0.5.1:
      Successfully uninstalled jaxlib-0.5.1
  Attempting uninstall: jax
    Found existing installation: jax 0.5.2
    Uninstalling jax-0.5.2:
      Successfully uninstalled jax-0.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have num

In [ ]:
# After restart: verify versions and try importing again
import numpy as np
print("NumPy version:", np.__version__)

from transformers import AutoProcessor, LlavaForConditionalGeneration
print("Transformers & LLaVA imports succeeded")

✅ NumPy version: 2.2.4


/usr/local/lib/python3.11/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.5.1 is installed, but it is not compatible with the installed jaxlib version 0.5.3, so it will not be used.
  warnings.warn(


✅ Transformers & LLaVA imports succeeded


In [17]:
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
import numpy as np
from itertools import product
from sklearn.metrics.pairwise import cosine_similarity
import joblib
from PIL import Image
from transformers import AutoProcessor, LlavaForConditionalGeneration
import torch

# Paths
base_path = "/content/drive/MyDrive/fashion_dataset"
image_base_path = f"{base_path}/data/"
cluster_csv = f"{base_path}/image_clusters.csv"
excel_path = f"{base_path}/data.xlsx"
test_data = f"{base_path}/test_anchor_images.csv"
model_path = f"{base_path}/rf_regressor_model.pkl"



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load LLaVA
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
llava_model = LlavaForConditionalGeneration.from_pretrained(
    "llava-hf/llava-1.5-7b-hf",
    torch_dtype=torch.float16,
    device_map="auto"
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [19]:
# Load data
df = pd.read_csv(cluster_csv)
df["filename"] = df["image_path"].apply(lambda x: os.path.basename(str(x).strip().replace("\\", "/")))
df_embed = df.set_index("filename")[[str(i) for i in range(512)]]
df_desc = pd.read_excel(excel_path)
desc_map = dict(zip(df_desc["main_image_url"].apply(lambda x: os.path.basename(str(x))), df_desc["description"]))
test_anchors = pd.read_csv(test_data)["test_anchors"].apply(lambda x: os.path.basename(str(x).strip().replace("\\", "/"))).tolist()
model = joblib.load(model_path)


In [ ]:
def get_llava_score(image_path, description):
    try:
        prompt = (
            "<image>\n"
            f"USER: The main item description is: {description}. "
            f"Please rate the full outfit visually from 1 to 10 based on compatibility.\nASSISTANT:"
        )
        image = Image.open(image_path).convert("RGB")
        inputs = processor(images=image, text=prompt, return_tensors="pt")

        inputs["pixel_values"] = inputs["pixel_values"].to("cuda", torch.float16)
        inputs["input_ids"] = inputs["input_ids"].to("cuda")
        inputs["attention_mask"] = inputs["attention_mask"].to("cuda")

        output = llava_model.generate(**inputs, max_new_tokens=20)
        result = processor.decode(output[0], skip_special_tokens=True)
        score = int("".join(filter(str.isdigit, result)))
        return min(max(score, 1), 10)
    except Exception as e:
        print(f"Error with LLaVA for {image_path}: {e}")
        return None


In [ ]:
# Build recommendations
cluster_dict = df.groupby("cluster")["filename"].apply(list).to_dict()
image_path_dict = dict(zip(df["filename"], df["image_path"]))

TOP_K = 3
recommendations = []

for anchor in test_anchors:
    print(f"🔍 Processing anchor: {anchor}")
    if anchor not in df_embed.index:
        print(f"Missing embedding for anchor: {anchor}")
        continue

    anchor_row = df[df["filename"] == anchor].iloc[0]
    anchor_cluster = anchor_row["cluster"]
    anchor_vec = df_embed.loc[anchor].values.reshape(1, -1)

    top_k_per_cluster = {}
    for c in cluster_dict:
        if c == anchor_cluster:
            continue
        candidates = cluster_dict[c]
        valid = [f for f in candidates if f in df_embed.index]
        if not valid:
            continue
        vecs = np.stack([df_embed.loc[f] for f in valid])
        sims = cosine_similarity(anchor_vec, vecs)[0]
        idx = np.argsort(sims)[::-1][:TOP_K]
        top_k_per_cluster[c] = [valid[i] for i in idx]

    if len(top_k_per_cluster) < 2:
        print(f"Skipping {anchor} — not enough clusters.")
        continue

    combos = list(product(*top_k_per_cluster.values()))
    best_score = -1
    best_outfit = None

    for combo in combos:
        try:
            vecs = [df_embed.loc[anchor]] + [df_embed.loc[x] for x in combo]
            full_vec = np.concatenate(vecs).reshape(1, -1)
            score = model.predict(full_vec)[0]
            if score > best_score:
                best_score = score
                best_outfit = [anchor] + list(combo)
        except Exception as e:
            print(f"Error scoring combo: {e}")
            continue

    if best_outfit:
        # ⬇get description for anchor
        anchor_file = os.path.basename(image_path_dict[best_outfit[0]].replace("\\", "/"))
        description = desc_map.get(anchor_file, "")

        print(f"Looking for description with key: {anchor_file}")
        print(f"Description: {description}")

        if not description or not isinstance(description, str) or description.strip() == "":
            print(f"No description for {anchor_file}, skipping...")
            continue

        from PIL import Image

        try:
            img = Image.open(image_path)
            img.verify()
            print(f"Image at {image_path} is valid.")
        except Exception as e:
            print(f"Image open failed: {image_path} | Error: {e}")
            continue




        description = desc_map.get(anchor_file, "")
        # fix path formatting for Colab
        relative = image_path_dict[best_outfit[0]].replace("\\", "/")
        image_path = os.path.join(base_path, relative)
        # score with LLaVA

        llava_val = get_llava_score(image_path, description)
        recommendations.append({
            "anchor": image_path_dict[best_outfit[0]],
            "match_1": image_path_dict[best_outfit[1]],
            "match_2": image_path_dict[best_outfit[2]],
            "model_score": round(best_score * 10, 2),
            "llava_score": llava_val
        })


🔍 Processing anchor: 3212_1.jpg
🎯 Looking for description with key: 3212_1.jpg
🧪 Description: Regular-fit overshirt in felted, woven fabric with a comfortable, classic silhouette. Collar, buttons at front, chest pockets with flap, and yoke at back. Long sleeves with button at cuffs. Rounded hem.
✅ Image at /content/drive/MyDrive/fashion_dataset/data/2262_1.jpg is valid.
🔍 Processing anchor: 1063_1.jpg
🎯 Looking for description with key: 1063_1.jpg
🧪 Description: Tailored, fitted vest in woven bouclé fabric. V-neck, buttons at front, darts at bust, and patch front pockets with button. Straight-cut hem with gently curved side vents. Lined.
✅ Image at /content/drive/MyDrive/fashion_dataset/data/3212_1.jpg is valid.
🔍 Processing anchor: 1322_1.jpg
🎯 Looking for description with key: 1322_1.jpg
🧪 Description: Short cardigan in structured jersey. Collar, metal buttons at front, and short sleeves finishing just above elbows.
✅ Image at /content/drive/MyDrive/fashion_dataset/data/1063_1.jpg is

In [ ]:
# Save results
df_result = pd.DataFrame(recommendations)
df_result.to_csv("llava_similarity_model_selected.csv", index=False)
print("Saved to llava_similarity_model_selected.csv")
if len(df_result):
    print(f"Average LLaVA Score: {np.mean([r for r in df_result['llava_score'] if pd.notnull(r)]):.2f}")

✅ Saved to llava_similarity_model_selected.csv
✅ Average LLaVA Score: 10.00
